In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# feature.txt 파일에는 피처 이름 index와 피처명이 공백으로 분리되어 있음. 이를 DataFrame으로 로드
feature_name_df = pd.read_csv("/Users/ijiseon/Desktop/data/human_activity/features.txt",
                             sep="\s+", header=None, names=["column_index","column_name"])

# 피처명 index를 제거하고, 피처명만 리스트 객체로 생성한 뒤 샘플로 10개만 추출
feautre_name= feature_name_df.iloc[:,1].values.tolist()
print("전체 피처명에서 10개만 추출:", feautre_name[:10])

전체 피처명에서 10개만 추출: ['tBodyAcc-mean()-X', 'tBodyAcc-mean()-Y', 'tBodyAcc-mean()-Z', 'tBodyAcc-std()-X', 'tBodyAcc-std()-Y', 'tBodyAcc-std()-Z', 'tBodyAcc-mad()-X', 'tBodyAcc-mad()-Y', 'tBodyAcc-mad()-Z', 'tBodyAcc-max()-X']


In [2]:
def get_new_feature_name_df (old_feature_name_df):
    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby("column_name").cumcount(), columns=["dup_cnt"])
    feature_dup_df  = feature_dup_df.reset_index()
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(),feature_dup_df, how="outer")
    new_feature_name_df["column_name"] = new_feature_name_df[["column_name",
                                                              "dup_cnt"]].apply(lambda x:x[0]+"_"+str(x[1])
                                                                               if x[1]>0 else x[0], axis=1)
    new_feature_name_df = new_feature_name_df.drop(["index"],axis=1)
    return new_feature_name_df

In [3]:
def get_human_dataset():
    
    #각 데이터 파일은 공백으로 분리되어 있으므로 read_csv에서 공백 문자를 sep으로 할당
    feature_name_df = pd.read_csv("/Users/ijiseon/Desktop/data/human_activity/features.txt", sep="\s+",
                                  header= None, names=["column_index","column_name"])
    
    # 중복된 피처명을 수정하는 get_new_feature_name_df()를 이용, 신규 피처명 DataFrame 생성
    new_feature_name_df = get_new_feature_name_df(feature_name_df)
    
    # DataFrame에 피처명을 칼럼으로 부여하기 위해 리스트 객체로 다시 변환
    feature_name = new_feature_name_df.iloc[:,1].values.tolist()
    
    #학습 피처 데이터세트와 테스트 피처 데이터를 DataFrame으로 로딩. 칼럼명은 feature_name 적용
    X_train = pd.read_csv("/Users/ijiseon/Desktop/data/human_activity/train/X_train.txt",sep="\s+", names=feature_name)
    X_test =pd.read_csv("/Users/ijiseon/Desktop/data/human_activity/test/X_test.txt",sep="\s+", names=feature_name)
    
    y_train= pd.read_csv("/Users/ijiseon/Desktop/data/human_activity/train/y_train.txt",sep="\s+", names=["action"])
    y_test= pd.read_csv("/Users/ijiseon/Desktop/data/human_activity/test/y_test.txt",sep="\s+", names=["action"])
    
    return X_train, X_test, y_train, y_test

In [4]:
from sklearn.ensemble import GradientBoostingClassifier
import time
import warnings

X_train, X_test, y_train, y_test = get_human_dataset()

In [6]:
from sklearn.metrics import accuracy_score

In [7]:
warnings.filterwarnings("ignore")

start_time = time.time()

gb_clf= GradientBoostingClassifier(random_state=0)
gb_clf.fit(X_train,y_train)
gb_pred = gb_clf.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)

print("GBM 정확도: {0:.4f}".format(gb_accuracy))
print("GBM 수행 시간: {0:.1f}초".format(time.time()-start_time))

GBM 정확도: 0.9386
GBM 수행 시간: 587.6초


In [8]:
from sklearn.model_selection import GridSearchCV

start_time = time.time()

params = {
    "n_estimators":[100,500],
    "learning_rate": [0.05,0.1]
}

grid_cv = GridSearchCV(gb_clf, param_grid=params, cv=2, verbose=1)
grid_cv.fit(X_train, y_train)

print("최적 하이퍼 파라미터:\n", grid_cv.best_params_)
print("최고 예측도: {0:.4f}".format(grid_cv.best_score_))
print("수행 시간: {0:.1f}초".format(time.time()-start_time))

Fitting 2 folds for each of 4 candidates, totalling 8 fits
최적 하이퍼 파라미터:
 {'learning_rate': 0.05, 'n_estimators': 500}
최고 예측도: 0.9004
수행 시간: 8352.6초


In [10]:
gb_pred = grid_cv.best_estimator_.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)
print("GBM 정확도: {0:.4f}".format(gb_accuracy))

GBM 정확도: 0.9393
